#SISTEMA DE RECOMENDAÇÃO - FILTRO COLABORATIVO
Durante  a  pandemia  os  números  de  vendas  da  Razzle  Dazzle  (e-commerce  de variedades) dispararam, bem como o da concorrência. Uma das maneiras de se 
destacar nesse mercado cada vez mais competitivo é oferecer o produto certo 
para a pessoa certa.
Então, você foi contratado desenvolver pelo menos um modelo de recomendação, 
que será disponibilizado no novo site da empresa. Além do modelo, é necessário 
informar qual o melhor momento para usá-lo, em propagandas, quando o cliente 
estiver fazendo uma pesquisa ou quando o cliente estiver vendo um produto. É 
fundamental que isso seja pensado no desenvolvimento do modelo.
 

IMPORTANDO OS PACOTES NECESSÁRIOS

In [2]:
from pathlib import Path
import progressbar as pb
import warnings # supress warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')


import pandas as pd
# Add some convenience functions to Pandas DataFrame.
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.3f}'.format

In [5]:
from google.colab import files
data1 = files.upload()

Saving DNC_order_items_dataset.csv to DNC_order_items_dataset.csv
Saving DNC_order_reviews_dataset.csv to DNC_order_reviews_dataset.csv
Saving DNC_orders_dataset.csv to DNC_orders_dataset (1).csv
Saving DNC_products_dataset.csv to DNC_products_dataset.csv


In [20]:
orders_df = pd.read_csv('DNC_orders_dataset.csv')
orders_items_df = pd.read_csv('DNC_order_items_dataset.csv')
reviews_df = pd.read_csv('DNC_order_reviews_dataset.csv')
products_df = pd.read_csv('DNC_products_dataset.csv')



In [50]:
orders_df.drop(columns = 'Unnamed: 0', inplace=True)
orders_df.head()

,order_id,customer_id,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered


In [51]:
orders_items_df.drop(columns = 'Unnamed: 0', inplace=True)
orders_items_df.head()

,order_id,order_item_id,product_id,price
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.900
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.900
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.000
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.990
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.900


In [52]:
reviews_df.drop(columns = 'Unnamed: 0', inplace=True)
reviews_df.head()

,review_id,order_id,review_score
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5


In [53]:
products_df.drop(columns = 'Unnamed: 0', inplace=True)
products_df.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.000,287.000,225.000,16.000,10.000,14.000
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.000,276.000,1000.000,30.000,18.000,20.000
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.000,250.000,154.000,18.000,9.000,15.000
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.000,261.000,371.000,26.000,4.000,26.000
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.000,402.000,625.000,20.000,17.000,13.000


In [54]:
print(orders_df.shape)
print(orders_items_df.shape)
print(reviews_df.shape)
print(products_df.shape)

(99441, 3)
(112650, 4)
(99224, 3)
(32951, 8)


#Filtrando





In [55]:
# filtrando somente pelos produtos entregues
orders_delivered_df = orders_df[orders_df['order_status'] =='delivered']
orders_delivered_df.head()

,order_id,customer_id,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered


In [56]:
# filtrando por número de itens de uma compra de no máximo 5 unidades
orders_items_ate_5_df = orders_items_df[orders_items_df['order_item_id'] < 6]
orders_items_ate_5_df.value_counts('order_item_id')

order_item_id
1    98666
2     9803
3     2287
4      965
5      460
dtype: int64

#Unindo tabelas

In [60]:
delivered_itens = orders_delivered_df.merge(orders_items_ate_5_df,on = "order_id", how = 'left')
delivered_itens_review = delivered_itens.merge(reviews_df, on = 'order_id', how = 'left')
delivered_itens_review.head()

,order_id,customer_id,order_status,order_item_id,product_id,price,review_id,review_score
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,1,87285b34884572647811a353c7ac498a,29.990,a54f0611adc9ed256b57ede6b6eb5114,4.000
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,1,595fac2a385ac33a80bd5114aec74eb8,118.700,8d5266042046a06655c8db133d120ba5,4.000
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,1,aa4383b373c6aca5d8797843e5594415,159.900,e73b67b67587f7644d5bd1a52deb1b01,5.000
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,1,d0b61bfb1de832b15ba9d266ca96e5b0,45.000,359d03e676b3c069f62cadba8dd3f6e8,5.000
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,1,65266b2da20d04dbe00c5c2d3bb7859e,19.900,e50934924e227544ba8246aeb3770dd4,5.000


#Recomendação de itens melhores avaliados

In [62]:
score_sum = pd.DataFrame(delivered_itens_review.groupby(['product_id'])['review_score'].sum())
top10 = score_sum.sort_values('review_score', ascending= False).head(10)
top10.merge(products_df,on ='product_id',how='left',left_index=True)


,product_id,review_score,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
13430,aca2eb7d00ea1a7b8ebd4e68314663af,2093.000,moveis_decoracao,44.000,903.000,2600.000,50.000,10.000,30.000
14051,422879e10f46682990de24d770e7f83d,1894.000,ferramentas_jardim,56.000,348.000,1550.000,30.000,22.000,30.000
9661,99a4788cb24856965c36a24e339b6058,1851.000,cama_mesa_banho,54.000,245.000,1383.000,50.000,10.000,40.000
30293,389d119b48cf3043d311335e499d9c6b,1601.000,ferramentas_jardim,59.000,341.000,1750.000,37.000,22.000,40.000
4598,368c6c730842d78016ad823897a372db,1522.000,ferramentas_jardim,57.000,366.000,1650.000,30.000,22.000,30.000
32098,53759a2ecddad2bb87a079a1f1519f73,1433.000,ferramentas_jardim,58.000,340.000,1500.000,30.000,22.000,30.000
8290,d1c427060a0f73f6b889a5c7c61f2ac4,1414.000,informatica_acessorios,59.000,1893.000,6550.000,20.000,20.000,20.000
793,53b36df67ebb7c41585e8d54d6772e08,1340.000,relogios_presentes,33.000,523.000,584.000,16.000,11.000,13.000
1749,154e7e31ebfa092203795c972e5804a6,1242.000,beleza_saude,48.000,575.000,100.000,20.000,15.000,15.000
29129,3dd2a17168ec895c781a9191c1e95ad7,1143.000,informatica_acessorios,48.000,894.000,533.000,22.000,11.000,20.000


In [68]:
products_df.dtypes

product_id                     object
product_category_name          object
product_name_lenght           float64
product_description_lenght    float64
product_weight_g              float64
product_length_cm             float64
product_height_cm             float64
product_width_cm              float64
dtype: object

In [70]:
delivered_itens_review.shape,delivered_itens_review['customer_id'].nunique(),delivered_itens_review['product_id'].nunique()

((110370, 8), 96478, 32199)

In [88]:
products_id = products_df['product_id'].nunique()

#1000 most active users
user_count = delivered_itens_review.groupby(by='customer_id', as_index=False) \
    .agg({'product_id': pd.Series.nunique}) \
    .rename(columns={'product_id':'product_count'})

shop_count = user_count.set_index('customer_id')
top1000_users = shop_count.sort_values('product_count', ascending = False).head(1000)

top_user_ids = top1000_users.index.tolist()
top_user_ids.sort()

In [89]:
sample = delivered_itens_review[delivered_itens_review['customer_id'].isin(top_user_ids)]
sample.shape

(2733, 8)

In [91]:
temp = pd.DataFrame({
    "user": sample["customer_id"],
    "product": sample["product_id"],
    "action": sample["review_score"]
})

result = temp.pivot_table(index="user", columns="product", values="action").fillna(0)
result

product,0011c512eb256aa0dbbb544d8dffcf6e,005030ef108f58b46b78116f754d8d38,008cff0e5792219fae03e570f980b330,00ba6d766f0b1d7b78a5ce3e1e033263,00faa46f36261af8bbf3a4d37fa4841b,0111ab6fb41da433e0f17d3f4ec2c8f5,0129d1e9b29d3fe6833cc922374cd252,01365901d8f2247976a7e5d4c98c52c0,014a8a503291921f7b004a5215bb3c36,0152f69b6cf919bcdaf117aa8c43e5a2,017692475c1c954ff597feda05131d73,01b0238705b6ae8e35c6fea837e6ae4a,01b660ebc1a0c293ccf9b117fa6dd8bb,01c2e91674406ebaca6a1bbf7f61c3f0,01d40fca05c4227b987ee069be837b49,01d4157fd9738ea81038316ae4cfc192,01e20e6604216c8adb31d463214ba00c,01ff1ff8aa5dec93e9938b989393a4ca,02475368dfb38934fe55f574024fe1d7,02623b0e4e6b951aefb23575de1f3886,0333795efd58e3c33c5850b92721c3e4,03566b85189259c49476f004ad2afd08,0369a47f7ad96df46ad8ed76767de8bd,03bb06cda40712fb8473f7962fb7d198,03d4233b9e3382adaed9e5c58626f1dc,03e931f387c2dd5b006aecdb12515b2e,04566e1a1afa83470b25c2642706fc32,045e41f597e72ebfe5e9de50788fbf03,04869307f7e7dc5b07f1d83406ade1e5,048bb098a0e194ddc3d0287b954bc9d8,049ac5dce8536884a652ce73c58022ab,04a67e69edfb208f784da30294f69c5b,04ca6bf4b3d9aa78046f0973efe2f4b1,04fd8d64a2539d188bef33bb288410bb,050d890ccc1c0874e166c9af9c6dd821,0522015abfd1094949180dee8dacbdf9,052413504e5295cd66ec4ced0b310062,055019def765ce08da9fac0585e927c3,056d012d264624accb7f73d31caee034,0573b564a90d2d9f9c460eb963da8c33,...,fb98e86cfbfbff699907f0a84784d474,fbaefac04950112a7da468bc016cc257,fbb63264ad9777810766ae4bed802224,fbc1488c1a1e72ba175f53ab29a248e8,fbce4c4cb307679d89a3bf3d3bb353b9,fbf19c12a50f1467bcae50364475ca26,fc04e723b53a7b372e081da67cf6aca2,fc1d8637c0268af3db482c14b7ef8e75,fc276546b539ea9ad62827c2a5e2ae42,fc5dd987f12a7b823a76a44aa1ba88f6,fc5feda8d6c915bdbd6ca84bac05e2eb,fc6b299cf83fa76632dd1b0a2c611f33,fc831da5307b1a3e23ed0259a2b6b2a1,fc8595227785863846b7daa2ae3f2c9b,fcf28afb1353f2f12ea041dd74954226,fcf39794919367e915d2656291f99564,fd0c762860528d119344e1377b574fc5,fd1061ebebbb9892c59897096066dec5,fd220f6fd5a3f13ef9db07428f9c935d,fd424396761d9e1609c5516b7d8ce17b,fd440fd792fd8a4b681f1eb258c25445,fd471a043ee8b8dd27f4086495e0724c,fd91e23b4465803725f329b39afe018a,fdba6f37e0258dbca54eb0ed8b293ae0,fde71f25e699ca0a2a83a6c3a249b816,fde90f06296b26995fd0de6f16610d70,fe59a1e006df3ac42bf0ceb876d70969,fe61253e50ad6ee9f1d180da22ce5eef,fe9c4b2cf9c3adbaf3644880fb72254d,fec094ad5af771b21cf38c6c9e946f19,fec3b45dc09b257690a09a742870b149,fec565c4e3ad965c73fb1a21bb809257,fef0dd8c53552d0f784b7cc5375a3ffe,fef4145d5309c82cd34bdfe608dd323c,ff285e6c96040323d3763dd184924018,ff2c1ec09b1bb340e84f0d6b21cc7dbb,ff55834c0b3272853dfe62340f2bb91b,ff6caf9340512b8bf6d2a2a6df032cfa,ff7a70b3356d433014b7d6957ed196cb,ff95ac47246ef13e48712ea1ff8df0d9
user,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
00796be07bfc87dad2e31ef15490a718,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
00b5072ef1192bc4d02e1d6264317bb0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
00ca6e4277e535d41bda84d371310be2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0

In [92]:
result.shape

(988, 2004)

In [104]:
from sklearn.metrics.pairwise import cosine_similarity
cos_item = cosine_similarity(result.T)

In [106]:
import numpy as np
import random
from pathlib import Path
import progressbar as pb
import warnings # supress warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')


import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
# Add some convenience functions to Pandas DataFrame.
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.3f}'.format

def mostra_n_mais_proximos(id_de_interesse, lista_instanciast, matriz_similaridade, n):
  print(f"Item de interesse: {instancia_de_interesse}")

  assert id_de_interesse in lista_instanciast
  cliente_idx = lista_instanciast.tolist().index(instancia_de_interesse)
  print(f"Item id: {instancia_de_interesse}, tem índice {cliente_idx}")

  closest_10_users = np.argsort(-matriz_similaridade[cliente_idx])[:n]
  print(instancia_de_interesse, closest_10_users)

  for i in zip(lista_instanciast[closest_10_users], matriz_similaridade[cliente_idx][closest_10_users]):
      print(f"Item {i[0]} tem similaridade {i[1]:.2f} com item {instancia_de_interesse}")


lista_instancias = result.T.index
instancia_de_interesse = lista_instancias[random.randint(0, len(lista_instancias))]

mostra_n_mais_proximos(instancia_de_interesse, lista_instancias, cos_item, 10)

Item de interesse: 0de59eddc63167215c972b0d785ffa7b
Item id: 0de59eddc63167215c972b0d785ffa7b, tem índice 103
0de59eddc63167215c972b0d785ffa7b [ 125  103  752   93 1498 1343 1342 1341 1340 1339]
Item 1065e0ebef073787a7bf691924c60eeb tem similaridade 1.00 com item 0de59eddc63167215c972b0d785ffa7b
Item 0de59eddc63167215c972b0d785ffa7b tem similaridade 1.00 com item 0de59eddc63167215c972b0d785ffa7b
Item 5dae498eff2d80057f56122235a36aff tem similaridade 1.00 com item 0de59eddc63167215c972b0d785ffa7b
Item 0cf2faf9749f53924cea652a09d8e327 tem similaridade 0.95 com item 0de59eddc63167215c972b0d785ffa7b
Item bd2c3709849255acdbf5855094cdaa01 tem similaridade 0.00 com item 0de59eddc63167215c972b0d785ffa7b
Item a9f480bfc49f0b85e8394e1cdf685b4e tem similaridade 0.00 com item 0de59eddc63167215c972b0d785ffa7b
Item a9eea144b709e6d670c9bf03ffc2dc2e tem similaridade 0.00 com item 0de59eddc63167215c972b0d785ffa7b
Item a996a4fecaea5e7e2638d2c5078ebf4b tem similaridade 0.00 com item 0de59eddc63167215c972b

#Criando módulos em Python para um Sistema de Recomendação

In [93]:
%%writefile data.py
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


def get_data(path):
  return pd.read_csv(path)


def start_pipeline(dataf):
  return dataf.copy()


def get_book_counts(dataf):
  return dataf.groupby(by='product_id', as_index=False) \
    .agg({'customer_id': pd.Series.nunique}) \
    .rename(columns={'customer_id': 'customer_id_count'}) \
    .set_index('product_id') \
    .sort_values('customer_id_count', ascending = False)


def get_n_top_values(dataf, n):
  return dataf.head(n) \
    .index \
    .tolist()


def format_sample(dataf):
  return pd.DataFrame({
      "user": dataf["customer_id"],
      "book": dataf["product_id"],
      "action": dataf["score_review"]
  })


def get_top_books_ids(dataf, sample_size=1000):
  return dataf \
    .pipe(start_pipeline) \
    .pipe(get_book_counts) \
    .pipe(get_n_top_values, sample_size)


def get_item_based_similarity_matrix(data):
  top_book_ids = get_top_books_ids(data)
  book_sampled_relationship_data = data[data["product_id"].isin(top_book_ids)] \
    .pipe(format_sample) \
    .pivot_table(index="user", columns="book", values="action").fillna(0)
  return cosine_similarity(book_sampled_relationship_data.T), book_sampled_relationship_data.T.index

Writing data.py


In [94]:
%%writefile item_based_recommender.py
import random

import numpy as np

from data import get_data, get_item_based_similarity_matrix


class ItemBasedRecommender:

  def __init__(self, data_path):
    self.data_path = data_path

  def setup(self):
    data = get_data(self.data_path)
    sim_matrix, available_items_index = get_item_based_similarity_matrix(data)
    self.available_items_index = available_items_index
    self.sim_matrix = sim_matrix

  def get_random_item_index(self):
    item_id = self.available_items_index[random.randint(0, self.sim_matrix.shape[0])]
    return item_id

  def get_n_closest(self, item_id, n=10):
    if item_id not in self.available_items_index:
      raise ValueError('Item is not available on data.')
    item_index = self.available_items_index.tolist().index(item_id)
    closest_n_iten = np.argsort(-self.sim_matrix[item_index])[1:n]
    n_closest = []
    for item_data in zip(self.available_items_index[closest_n_iten], self.sim_matrix[item_index][closest_n_iten]):
      n_closest.append(item_data)
    return tuple(n_closest)

Writing item_based_recommender.py


In [96]:
from item_based_recommender import ItemBasedRecommender

item_recommender = ItemBasedRecommender(delivered_itens_review)
item_recommender.setup()

ValueError: ignored